In [1]:
#Include Libraries
import pandas as pd
import pickle
import re
import numpy as np

In [2]:
pd.options.mode.chained_assignment = None 
with open('save_house_chotot.pickle','rb') as handle:
    dataset = pickle.load(handle)
dataset.head()

,Location,Price,Name,Date,Feature0,Feature1,Feature3,Feature2
0,"17, Cẩm Băc 8, Phường Hòa Thọ Đông, Quận Cẩm L...","1,6 triệu/tháng- 20 m2","Phòng trọ Cẩm Lệ, rộng rãi, sạch đẹp an ninh!",Tin cá nhân đăng 2 tuần trước,Cho thuê,Diện tích: 20 m²,Số tiền cọc: 500.000 đ,NaN
1,"132, Ba Tháng Hai, Phường Thuận Phước, Quận Hả...","1,1 triệu/tháng- 40 m2",Trọ homestay-1.1tr-Điều hoà-Bao điện nước,Tin cá nhân đăng 3 tuần trước,Cho thuê,Diện tích: 40 m²,Số tiền cọc: 50.000 đ,Tình trạng nội thất: Nội thất đầy đủ
2,"K1120/58 Trường Chinh, Phường Hòa Phát, Quận C...","1,5 triệu/tháng- 20 m2",Phòng trọ Quận Cẩm Lệ 20m²,Tin cá nhân đăng 4 tháng trước,Cho thuê,Diện tích: 20 m²,Số tiền cọc: 500.000 đ,Tình trạng nội thất: Nhà trống
3,"Trần Đăng, Phường Hòa Minh, Quận Liên Chiểu, Đ...","2,5 triệu/tháng- 25 m2","Phòng trọ Cao cấp25m² Kinh Dương Vương,Liên Chiểu",Tin cá nhân đăng 1 giờ trước,Cho thuê,Diện tích: 25 m²,Số tiền cọc: 2.500.000 đ,Tình trạng nội thất: Nội thất đầy đủ
4,"Cao Xuân Duc, Phường Thuận Phước, Quận Hải Châ...","1,2 triệu/tháng- 30 m2",trọ homestay,Tin cá nhân đăng 2 giờ trước,Cho thuê,Diện tích: 30 m²,Số tiền cọc: 1.000.000 đ,Tình trạng nội thất: Nội thất đầy đủ


In [3]:
reg = "(\d+)\W*(\d*) (triệu|đ)"
x = "500000 đ"
result = re.findall(reg,x)
print(result)

[('500000', '', 'đ')]


In [4]:
#Regex for extracting useful information
price_re = "[(\d\W)*\d+]+(triệu|tỷ|đ)+"
price_stn = "(\d+)\W*(\d*) (triệu|đ)"
location_re = "[Phường|Xã]+((\W\w+)+), [Quận|Huyện]+((\W\w+)+)"
area_re = "\d+"
deposit_fee = "\d+"
furniture_re = "Nội thất đầy đủ|Nhà trống"

#Modifying the dataframe
dataset['Price'] = dataset['Price'].apply(lambda x: re.match(price_re,x)[0])
dataset = dataset[dataset['Price'].str.contains("tỷ") == False]
dataset = dataset.reset_index(drop = True)
dataset['Ward'] = dataset['Location'].apply(lambda x: list(re.findall(location_re,x)[0])[0])
dataset['District'] = dataset['Location'].apply(lambda x: list(re.findall(location_re,x)[0])[2])
dataset['Feature1'] = dataset['Feature1'].apply(lambda x: re.findall(area_re,x)[0])
for i in range(len(dataset['Feature3'])):
    try:
        dataset['Feature3'][i] = int(''.join(re.findall(deposit_fee,dataset['Feature3'][i])))
    except: pass
for i in range(len(dataset['Feature2'])):
    try:
        dataset['Feature2'][i] = re.findall(furniture_re,dataset['Feature2'][i])[0]
    except: pass
for i in range(len(dataset['Price'])):
    price_components = list(re.findall(price_stn,dataset['Price'][i])[0])
    if (price_components[2] == 'triệu'):
        if (price_components[1] != ''):
            dataset['Price'][i] = int(float(price_components[0]) * 1000000 + float(str(0.) + price_components[1]) * 10000000)
        else: dataset['Price'][i] = int(float(price_components[0]) * 1000000)
    else:
        dataset['Price'][i] = int(float(''.join([price_components[0],price_components[1]])))
dataset.head()

,Location,Price,Name,Date,Feature0,Feature1,Feature3,Feature2,Ward,District
0,"17, Cẩm Băc 8, Phường Hòa Thọ Đông, Quận Cẩm L...",1600000,"Phòng trọ Cẩm Lệ, rộng rãi, sạch đẹp an ninh!",Tin cá nhân đăng 2 tuần trước,Cho thuê,20,500000,NaN,Hòa Thọ Đông,Cẩm Lệ
1,"132, Ba Tháng Hai, Phường Thuận Phước, Quận Hả...",1100000,Trọ homestay-1.1tr-Điều hoà-Bao điện nước,Tin cá nhân đăng 3 tuần trước,Cho thuê,40,50000,Nội thất đầy đủ,Thuận Phước,Hải Châu
2,"K1120/58 Trường Chinh, Phường Hòa Phát, Quận C...",1500000,Phòng trọ Quận Cẩm Lệ 20m²,Tin cá nhân đăng 4 tháng trước,Cho thuê,20,500000,Nhà trống,Hòa Phát,Cẩm Lệ
3,"Trần Đăng, Phường Hòa Minh, Quận Liên Chiểu, Đ...",2500000,"Phòng trọ Cao cấp25m² Kinh Dương Vương,Liên Chiểu",Tin cá nhân đăng 1 giờ trước,Cho thuê,25,2500000,Nội thất đầy đủ,Hòa Minh,Liên Chiểu
4,"Cao Xuân Duc, Phường Thuận Phước, Quận Hải Châ...",1200000,trọ homestay,Tin cá nhân đăng 2 giờ trước,Cho thuê,30,1000000,Nội thất đầy đủ,Thuận Phước,Hải Châu


In [5]:
dataset = dataset.drop(columns = ['Location', 'Name', 'Date','Feature0'])
dataset = dataset.rename(columns = {"Feature1" :"Area", "Feature3" :"Deposit Fee", "Feature2" : "Furniture"})
dataset.head()

,Price,Area,Deposit Fee,Furniture,Ward,District
0,1600000,20,500000,NaN,Hòa Thọ Đông,Cẩm Lệ
1,1100000,40,50000,Nội thất đầy đủ,Thuận Phước,Hải Châu
2,1500000,20,500000,Nhà trống,Hòa Phát,Cẩm Lệ
3,2500000,25,2500000,Nội thất đầy đủ,Hòa Minh,Liên Chiểu
4,1200000,30,1000000,Nội thất đầy đủ,Thuận Phước,Hải Châu


In [6]:
dataset.to_csv('Modified_Dataset.csv')